# AIモデルの評価: コード、ヒューマン、モデルベースの採点

このノートブックでは、Claude v3のようなAIモデルの効果を評価するために広く使用されている3つの手法について詳しく見ていきます。

1. コードベースの採点
2. ヒューマン採点
3. モデルベースの採点

それぞれのアプローチを例を通じて示し、AIのパフォーマンスを評価する際のそれぞれの利点と制限について検討します。

## コードベースの採点例: 感情分析

この例では、Claudeの映画レビューの感情をポジティブまたはネガティブに分類する能力を評価します。モデルの出力が期待される感情と一致するかどうかを確認するためにコードを使用できます。

In [ ]:
# pythonの組み込み正規表現ライブラリをインポート
import re

# boto3とjsonをインポート
import boto3
import json
from botocore.exceptions import ClientError
# 後で使用するためにモデル名とAWSリージョンを保存

%store -r modelId
%store -r region

In [ ]:
# Function to build the input prompt for sentiment analysis
def build_input_prompt(review):
    user_content = f"""Classify the sentiment of the following movie review as either 'positive' or 'negative' provide only one of those two choices:
    <review>{review}</review>"""
    return [{"role": "user", "content": [{"text": user_content}]}]

# Define the evaluation data
eval = [
    {
        "review": "This movie was amazing! The acting was superb and the plot kept me engaged from start to finish.",
        "golden_answer": "positive"
    },
    {
        "review": "I was thoroughly disappointed by this film. The pacing was slow and the characters were one-dimensional.",
        "golden_answer": "negative"
    }
]

# Function to get completions from the model
bedrock_client = boto3.client(service_name='bedrock-runtime', region_name=region)

def get_completion(messages, system_prompt=None):
    inference_config = {
        "temperature": 0.5,
        "maxTokens": 200
    }
    additional_model_fields = {
        "top_p": 1
    }
    converse_api_params = {
        "modelId": modelId,
        "messages": messages,
        "inferenceConfig": inference_config,
        "additionalModelRequestFields": additional_model_fields
    }
    if system_prompt:
        converse_api_params["system"] = [{"text": system_prompt}]
    try:
        response = bedrock_client.converse(**converse_api_params)
        text_content = response['output']['message']['content'][0]['text']
        return text_content

    except ClientError as err:
        message = err.response['Error']['Message']
        print(f"A client error occured: {message}")

# Get completions for each input
outputs = [get_completion(build_input_prompt(item["review"])) for item in eval]

# Print the outputs and golden answers
for output, question in zip(outputs, eval):
    print(f"Review: {question['review']}\nGolden Answer: {question['golden_answer']}\nOutput: {output}\n")

# Function to grade the completions
def grade_completion(output, golden_answer):
    return output.lower() == golden_answer.lower()

# Grade the completions and print the accuracy
grades = [grade_completion(output, item["golden_answer"]) for output, item in zip(outputs, eval)]
print(f"Accuracy: {sum(grades) / len(grades) * 100}%")

## 人間の採点例：エッセイのスコアリング

エッセイのスコアリングのようなタスクは、コードだけで評価するのが難しいです。この場合、モデルの出力を評価するための人間の採点者向けのガイドラインを提供できます。

In [ ]:
# Function to build the input prompt for essay generation
def build_input_prompt(topic):
    user_content = f"""Write a short essay discussing the following topic:
    <topic>{topic}</topic>"""
    return [{"role": "user", "content": [{"text": user_content}]}]

# Define the evaluation data
eval = [
    {
        "topic": "The importance of education in personal development and societal progress",
        "golden_answer": "A high-scoring essay should have a clear thesis, well-structured paragraphs, and persuasive examples discussing how education contributes to individual growth and broader societal advancement."
    }
]

# Get completions for each input
outputs = [get_completion(build_input_prompt(item["topic"])) for item in eval]

# Print the outputs and golden answers
for output, item in zip(outputs, eval):
    print(f"Topic: {item['topic']}\n\nGrading Rubric:\n {item['golden_answer']}\n\nModel Output:\n{output}\n")

## モデルベースの採点例

私たちは、モデルの応答と採点基準を提供することで、Claudeが自分自身の出力を採点することができます。これにより、通常は人間の判断を必要とするタスクの評価を自動化することが可能になります。

### 例 1: 要約

この例では、Claudeを使用して生成した要約の質を評価します。これは、モデルが長いテキストから重要な情報を簡潔かつ正確に捉える能力を評価する必要があるときに役立ちます。カバーすべき重要なポイントを概説したルーブリックを提供することで、採点プロセスを自動化し、要約タスクにおけるモデルのパフォーマンスを迅速に評価できます。

In [ ]:
# Function to build the input prompt for summarization
def build_input_prompt(text):
    user_content = f"""Please summarize the main points of the following text:
    <text>{text}</text>"""
    return [{"role": "user", "content": [{"text": user_content}]}]

# Function to build the grader prompt for assessing summary quality
def build_grader_prompt(output, rubric):
    user_content = f"""Assess the quality of the following summary based on this rubric:
    <rubric>{rubric}</rubric>
    <summary>{output}</summary>
    Provide a score from 1-5, where 1 is poor and 5 is excellent."""
    return [{"role": "user", "content": [{"text": user_content}]}]

# Define the evaluation data
eval = [
    {
        "text": "The Magna Carta, signed in 1215, was a pivotal document in English history. It limited the powers of the monarchy and established the principle that everyone, including the king, was subject to the law. This laid the foundation for constitutional governance and the rule of law in England and influenced legal systems worldwide.",
        "golden_answer": "A high-quality summary should concisely capture the key points: 1) The Magna Carta's significance in English history, 2) Its role in limiting monarchical power, 3) Establishing the principle of rule of law, and 4) Its influence on legal systems around the world."
    }
]

# Get completions for each input
outputs = [get_completion(build_input_prompt(item["text"])) for item in eval]

# Grade the completions
grades = [get_completion(build_grader_prompt(output, item["golden_answer"])) for output, item in zip(outputs, eval)]

# Print the summary quality score
print(f"Summary quality score: {grades[0]}")

### 例2: ファクトチェック

この例では、Claudeを使用して主張をファクトチェックし、そのファクトチェックの正確性を評価します。これは、モデルが正確な情報と不正確な情報を区別する能力を評価する必要があるときに役立ちます。正しいファクトチェックでカバーすべき重要なポイントを概説したルーブリックを提供することで、採点プロセスを自動化し、ファクトチェックタスクにおけるモデルのパフォーマンスを迅速に評価できます。

In [ ]:
# Function to build the input prompt for fact-checking
def build_input_prompt(claim):
    user_content = f"""Determine if the following claim is true or false:
    <claim>{claim}</claim>"""
    return [{"role": "user", "content": [{"text": user_content}]}]

# Function to build the grader prompt for assessing fact-check accuracy
def build_grader_prompt(output, rubric):
    user_content = f"""Based on the following rubric, assess whether the fact-check is correct:
    <rubric>{rubric}</rubric>
    <fact-check>{output}</fact-check>"""
    return [{"role": "user", "content": [{"text": user_content}]}]

# Define the evaluation data
eval = [
    {
        "claim": "The Great Wall of China is visible from space.",
        "golden_answer": "A correct fact-check should state that this claim is false. While the Great Wall is an impressive structure, it is not visible from space with the naked eye."
    }
]

# Get completions for each input
outputs = [get_completion(build_input_prompt(item["claim"])) for item in eval]

grades = []
for output, item in zip(outputs, eval):
    # 主張、ファクトチェック、およびルブリックを出力します
    print(f"Claim: {item['claim']}\n")
    print(f"Fact-check: {output}]\n")
    print(f"Rubric: {item['golden_answer']}\n")
    
    # ファクトチェックを評価します
    grader_prompt = build_grader_prompt(output, item["golden_answer"])
    grade = get_completion(grader_prompt)
    grades.append("correct" in grade.lower())

# ファクトチェックの正確性を出力します
accuracy = sum(grades) / len(grades)
print(f"Fact-checking accuracy: {accuracy * 100}%")

### 例 3: トーン分析

この例では、Claudeを使用して与えられたテキストのトーンを分析し、その分析の正確性を評価します。これは、モデルがテキストの中で表現された感情的な内容や態度を特定し解釈する能力を評価する必要があるときに役立ちます。特定すべきトーンの重要な側面を概説したルーブリックを提供することで、採点プロセスを自動化し、トーン分析タスクにおけるモデルのパフォーマンスを迅速に評価できます。

In [ ]:
# Function to build the input prompt for tone analysis
def build_input_prompt(text):
    user_content = f"""以下のテキストのトーンを分析してください:
    <text>{text}</text>"""
    return [{"role": "user", "content": [{"text": user_content}]}]

# Function to build the grader prompt for assessing tone analysis accuracy
def build_grader_prompt(output, rubric):
    user_content = f"""このルブリックに基づいて、以下のトーン分析の正確性を評価してください:
    <rubric>{rubric}</rubric>
    <tone-analysis>{output}</tone-analysis>"""
    return [{"role": "user", "content": [{"text": user_content}]}]

# Define the evaluation data
eval = [
    {
        "text": "イベントが直前にキャンセルされたなんて信じられません。これは完全に受け入れられず、非専門的です！",
        "golden_answer": "トーン分析は、テキストがフラストレーション、怒り、失望を表現していることを特定する必要があります。'信じられない'、'直前'、'受け入れられない'、'非専門的'のようなキーワードは、強い否定的感情を示しています。"
    }
]

# Get completions for each input
outputs = [get_completion(build_input_prompt(item["text"])) for item in eval]

# Grade the completions
grades = [get_completion(build_grader_prompt(output, item["golden_answer"])) for output, item in zip(outputs, eval)]

# Print the tone analysis quality
print(f"トーン分析の質: {grades[0]}")

これらの例は、`Claude`のようなAIモデルをさまざまなタスクで評価するために、コードベース、人間、モデルベースの採点がどのように使用できるかを示しています。評価方法の選択は、タスクの性質と利用可能なリソースに依存します。モデルベースの採点は、人間の判断を必要とする複雑なタスクの評価を自動化するための有望なアプローチを提供します。